In [17]:
devtools::install_github("immunogenomics/singlecellmethods", auth_token = "c3d2b3349a38a60ef5a3edc8cb2199350312592c")


✔  checking for file ‘/tmp/Rtmp5RveiH/remotesc9632d589451/immunogenomics-singlecellmethods-071799e2e1dcd5a6da9010f2335519a233b3aa58/DESCRIPTION’
─  preparing ‘singlecellmethods’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘singlecellmethods_0.1.0.tar.gz’
   


ERROR: Error in i.p(...): (converted from warning) installation of package ‘/tmp/Rtmp5RveiH/filec96362e2cbe5/singlecellmethods_0.1.0.tar.gz’ had non-zero exit status


In [ ]:
library(Rcpp)
library(tidyverse)
library(data.table)
library(magrittr)
library(harmony)
library(singlecellmethods)

In [7]:
normalizeData <- function(A, scaling_factor = 1e4, do_ftt = FALSE) {
	A@x <- A@x / rep.int(Matrix::colSums(A), diff(A@p))
    A@x <- scaling_factor * A@x
    if (do_ftt) {
        A@x <- sqrt(A@x) + sqrt(1 + A@x)
    } else {
        A@x <- log(1 + A@x)
    }
	return(A)
}

In [8]:
# X <- Matrix::t(harmony::ctrl.sparse)
X <- Matrix::cbind2(harmony::ctrl.sparse, harmony::stim.sparse)
X %<>% normalizeData()
X_ctrl <- normalizeData(harmony::ctrl.sparse)
X_stim <- normalizeData(harmony::stim.sparse)

In [2]:
findVariableGenes <- function(X, groups, min_expr = .1, max_expr = Inf, 
                              min_dispersion = 0, max_dispersion = Inf, return_top_n = 0) {
    ## TODO: check that groups are 0 indexed
    groups <- factor(groups)
    groups_int <- as.integer(factor(groups)) - 1
    groups_table <- table(groups_int)
    
    ## initially compute means in non-log space, to use in vmr function below
    means_nonlog <- exp_mean(X@x, X@p, X@i, ncol(X), nrow(X), groups_int, groups_table)
    colnames(means_nonlog) <- levels(groups)
    row.names(means_nonlog) <- row.names(X)
    
    vmr <- log_vmr(X@x, X@p, X@i, ncol(X), nrow(X), means_nonlog, groups_int, groups_table)    
    colnames(vmr) <- levels(groups)
    row.names(vmr) <- row.names(X)
    
    ## transform means to logspace and join means and VMR  
    vargenes_df <- dplyr::inner_join(
        means_nonlog %>% log1p %>% as.data.frame() %>% 
            tibble::rownames_to_column("symbol") %>% 
            tidyr::gather(group, gene_mean, -symbol),
        vmr %>% as.data.frame() %>% 
            tibble::rownames_to_column("symbol") %>% 
            tidyr::gather(group, gene_dispersion, -symbol), 
        by = c("symbol", "group")
    ) %>% 
        dplyr::arrange(-gene_dispersion) %>% 
        subset(gene_mean >= min_expr & gene_mean <= max_expr) %>%
        subset(gene_dispersion >= min_dispersion & gene_dispersion <= max_dispersion)    
    
    if (return_top_n > 0) {
        vargenes_union <- unique(data.table(vargenes_df)[, head(.SD, return_top_n), by = group][, symbol])
        return(vargenes_union)
    } else {
        return(vargenes_df)
    }
    
}



In [16]:
system.time({
    .res <- findVariableGenes(X, c(rep("ctrl", ncol(harmony::ctrl.sparse)), rep("stim", ncol(harmony::stim.sparse))))
#     .res <- findVariableGenes(X_ctrl, rep(0, ncol(X_ctrl)))
})

.res %>% head

   user  system elapsed 
  1.033   0.124   1.119 

symbol,group,gene_mean,gene_dispersion
HBB,stim,4.1808174,8.741350
HBB,ctrl,4.0482790,8.721870
IGLL5,stim,0.7199632,7.981912
HBA2,ctrl,2.7746720,7.428169
HBA2,stim,2.8167332,7.379681
HBG1,ctrl,0.4015790,7.132339
